In [1]:
import pandas as pd
import pickle
import numpy as np
import pathlib

In [2]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
clean_data_path = DATA_DIR / 'processed' / 'ames_clean.pkl'

In [3]:
with open(clean_data_path, 'rb') as f:
    data = pickle.load(f)

In [4]:
data.head()

,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Lot.Shape,Land.Contour,Lot.Config,Land.Slope,Neighborhood,Bldg.Type,...,Sale.Type,Sale.Condition,SalePrice,Condition,HasShed,HasAlley,Exterior,Garage.Age,Remod.Age,House.Age
0,20,RL,141.0,31770.0,IR1,Lvl,Corner,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.332438,Norm,False,False,BrkFace,50.0,50.0,50.0
1,20,RH,80.0,11622.0,Reg,Lvl,Inside,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.021189,Roads,False,False,VinylSd,49.0,49.0,49.0
2,20,RL,81.0,14267.0,IR1,Lvl,Corner,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.235528,Norm,False,False,Wd Sdng,52.0,52.0,52.0
3,20,RL,93.0,11160.0,Reg,Lvl,Corner,Gtl,NAmes,1Fam,...,GroupedWD,Normal,5.387390,Norm,False,False,BrkFace,42.0,42.0,42.0
4,60,RL,74.0,13830.0,IR1,Lvl,Inside,Gtl,Gilbert,1Fam,...,GroupedWD,Normal,5.278525,Norm,False,False,VinylSd,13.0,12.0,13.0


In [5]:
model_df = data.copy()

In [6]:
categorical_columns = []
ordinal_columns = []
for col in model_df.select_dtypes('category').columns:
    if model_df[col].cat.ordered:
        ordinal_columns.append(col)
    else:
        categorical_columns.append(col)

In [7]:
for col in ordinal_columns:
    codes, _ = pd.factorize(data[col], sort=True)
    model_df[col] = codes

In [8]:
original_data = model_df['Exterior']
encoded_data = pd.get_dummies(original_data)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
AsbShng,False,False,False,False,False
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False


In [9]:
original_data = model_df['Exterior']
encoded_data = pd.get_dummies(original_data, drop_first=True)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False
Other,False,False,False,False,False


In [10]:
model_df = pd.get_dummies(model_df, drop_first=True)

In [11]:
for cat in categorical_columns:
    dummies = []
    for col in model_df.columns:
        if col.startswith(cat + "_"):
            dummies.append(f'"{col}"')
    dummies_str = ', '.join(dummies)
    print(f'From column "{cat}" we made {dummies_str}\n')

From column "MS.SubClass" we made "MS.SubClass_30", "MS.SubClass_50", "MS.SubClass_60", "MS.SubClass_70", "MS.SubClass_80", "MS.SubClass_85", "MS.SubClass_90", "MS.SubClass_120", "MS.SubClass_160", "MS.SubClass_190", "MS.SubClass_Other"

From column "MS.Zoning" we made "MS.Zoning_RH", "MS.Zoning_RL", "MS.Zoning_RM"

From column "Land.Contour" we made "Land.Contour_HLS", "Land.Contour_Low", "Land.Contour_Lvl"

From column "Lot.Config" we made "Lot.Config_CulDSac", "Lot.Config_FR2", "Lot.Config_FR3", "Lot.Config_Inside"

From column "Neighborhood" we made "Neighborhood_BrDale", "Neighborhood_BrkSide", "Neighborhood_ClearCr", "Neighborhood_CollgCr", "Neighborhood_Crawfor", "Neighborhood_Edwards", "Neighborhood_Gilbert", "Neighborhood_IDOTRR", "Neighborhood_MeadowV", "Neighborhood_Mitchel", "Neighborhood_NAmes", "Neighborhood_NPkVill", "Neighborhood_NWAmes", "Neighborhood_NoRidge", "Neighborhood_NridgHt", "Neighborhood_OldTown", "Neighborhood_SWISU", "Neighborhood_Sawyer", "Neighborhood_Sa

In [12]:
from sklearn.model_selection import train_test_split

X = model_df.drop('SalePrice', axis=1)
y = model_df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [85]:
from xgboost import XGBRegressor

modelo = XGBRegressor(objective="reg:squarederror",n_jobs=-1,n_estimators=1000,max_depth=3,learning_rate=0.1)

modelo.fit(X_train, y_train)

y_hat = modelo.predict(X_test)

/home/eduardo/Desktop/machine-learning/Machine-Learning-Ames/env/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/eduardo/Desktop/machine-learning/Machine-Learning-Ames/env/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/eduardo/Desktop/machine-learning/Machine-Learning-Ames/env/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/eduardo/Desktop/machine-learning/Machine-Learning-Ames/env/lib/python3.10/site-packages/xgboost/data.py:359: Future

In [86]:
from sklearn.metrics import mean_squared_error


error = mean_squared_error(y_hat, y_test)
error = np.sqrt(error)
error

0.04705243946415031

In [87]:
print(f"Average error is {100*(10**error - 1):.2f}%")

Average error is 11.44%


In [16]:
from lightgbm import LGBMRegressor

modelo = LGBMRegressor(objective="regression", n_jobs=-1)

modelo.fit(X_train, y_train)

y_hat = modelo.predict(X_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007235 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3662
[LightGBM] [Info] Number of data points in the train set: 2301, number of used features: 151
[LightGBM] [Info] Start training from score 5.224001


In [17]:
error = mean_squared_error(y_hat, y_test)

error = np.sqrt(error)

error

0.05031035273460047

In [18]:
print(f"Average error is {100*(10**error - 1):.2f}%")

Average error is 12.28%


In [64]:
from sklearn.svm import SVR

modelo = SVR(kernel="linear", C=1e5, epsilon=0.2, max_iter=100000)

modelo.fit(X_train, y_train)

y_hat = modelo.predict(X_test)

/home/eduardo/Desktop/machine-learning/Machine-Learning-Ames/env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [65]:
error = mean_squared_error(y_hat, y_test)

error = np.sqrt(error)

error

0.1993504684096772

In [66]:
print(f"Average error is {100*(10**error - 1):.2f}%")

Average error is 58.25%


In [73]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(n_neighbors=10)

model.fit(X_train, y_train)

yhat = model.predict(X_test)

In [74]:
error = mean_squared_error(yhat, y_test)

error = np.sqrt(error)

print(f"Average error is {100*(10**error - 1):.2f}%")

Average error is 22.65%


In [75]:
from sklearn.linear_model import BayesianRidge

model = BayesianRidge()

model.fit(X_train, y_train)

yhat = model.predict(X_test)

error = np.sqrt(mean_squared_error(yhat, y_test))

print(f"Average error is {100*(10**error - 1):.2f}%")

Average error is 14.70%
